In [2]:
import numpy as np

In [106]:
def NaiveGaussinElimination(M_:np.ndarray, b_:np.ndarray, printX=False):
    M = M_.copy()
    b = b_.copy()
    if M.shape[0] != M.shape[1]:
        print("M is not a square matrix.")
        return None

    order = M.shape[0]

    # Elimination
    for col in range(order):
        for row in range(col+1, order):
            mult = M[row][col]/M[col][col]
            for k in range(col, order):
                M[row][k] -= mult*M[col][k]
            b[row] -= mult*b[col]

    # Back substitution
    res = np.zeros(order)
    for row in range(order)[::-1]:
        for col in range(row+1, order):
            b[row] -= M[row][col]*res[col]
        res[row] = b[row]/M[row][row]

    if printX:
        print('x:\n', res)
    return res

1. 对于元素为 $A_{ij}=5/(i+2j-1)$ 的 $n\times n$ 矩阵，令 $x=[1,\,\dots\,,1]^T, \;b=Ax$. 使用2.1.1的程序（高斯消元法）计算双精度解 $x_c$. 找出 $Ax=b$ 的前向误差和误差放大因子的无穷范数，并于A的条件数进行比较: $(a)\;n=6,\quad(b)\;n=10$ .

In [58]:
from numpy.linalg import norm as npnorm

def Solve(matrixGen, n, printX=True):
    print(f'\nn={n}')
    A = matrixGen(n)
    b = A@np.ones(n)
    x = NaiveGaussinElimination(A, b, printX)
    BE_norm = npnorm(A@x-b, np.inf)
    FE_norm = npnorm(x-np.ones(n), np.inf)
    print(f'BE_norm: {BE_norm:.5e}')
    print(f'FE_norm: {FE_norm:.5e}')
    print(f'Error magnification factor: {FE_norm/1/(BE_norm/np.max(np.abs(b))):.5e}')
    print(f'Condition number: {npnorm(A, np.inf)*npnorm(np.linalg.inv(A), np.inf):.5e}')

In [68]:
makeA = lambda n: np.array([[5./(row+2*col-1) for col in range(1, n+1)] for row in range(1, n+1)])
Solve(makeA, 6)


n=6
x:
 [1. 1. 1. 1. 1. 1.]
BE_norm: 8.88178e-16
FE_norm: 6.25191e-10
Error magnification factor: 4.31140e+06
Condition number: 7.03420e+07


In [48]:
Solve(makeA, 10)


n=10
x:
 [1.         1.00000003 0.99999896 1.00001421 0.99990825 1.00031949
 0.99936593 1.00071684 0.99957066 1.00010564]
BE_norm: 8.88178e-16
FE_norm: 7.16845e-04
Error magnification factor: 5.90989e+12
Condition number: 1.31370e+14


2. 对于 $A_{ij}=1/(|i-j|+1)$ 的矩阵解答1中的问题。

In [29]:
def MakeA2(n):
    return np.array([
        [
            1./(np.abs(row-col)+1) for col in range(1, n+1)
        ] for row in range(1, n+1)
    ], dtype=np.float64)

In [49]:
Solve(MakeA2, 6)


n=6
x:
 [1. 1. 1. 1. 1. 1.]
BE_norm: 4.44089e-16
FE_norm: 3.33067e-16
Error magnification factor: 2.18750e+00
Condition number: 8.61168e+00


In [50]:
Solve(MakeA2, 10)


n=10
x:
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
BE_norm: 8.88178e-16
FE_norm: 1.11022e-15
Error magnification factor: 4.66667e+00
Condition number: 1.12612e+01


3. 对于元素为 $A_{ij}=|i-j|+1$ 的 $n\times n$ 矩阵，解答1中的问题，其中 $n=100,\;200,\;300,\;400,\;500$. 并找出问题 $Ax=b$ 的5个误差放大因子，并和对应的条件数进行比较。


In [52]:
makeA3 = lambda n: np.array([[np.abs(row-col)+1. for row in range(1, n+1)] for col in range(1, n+1)])

In [65]:
def FindOtherEMF(matrixGen, n):
    A = matrixGen(n)
    for i in range(2, 6):
        xa = np.ones(n)*i
        b = A@xa
        x = NaiveGaussinElimination(A, b, False)
        BE_norm = npnorm(A@x-b, np.inf)
        FE_norm = npnorm(x-xa, np.inf)
        print(f'EMF with b={i}: {FE_norm/i/(BE_norm/np.max(np.abs(b))):.5e}')

In [66]:
for i in range(1, 6):
    Solve(makeA3, i*100, False)
    FindOtherEMF(makeA3, i*100)


n=100
BE_norm: 2.13731e-10
FE_norm: 5.29112e-11
Error magnification factor: 1.25018e+03
Condition number: 1.01000e+04
EMF with b=2: 1.25018e+03
EMF with b=3: 1.44542e+03
EMF with b=4: 1.25018e+03
EMF with b=5: 6.99530e+02

n=200
BE_norm: 1.84809e-09
FE_norm: 5.78179e-10
Error magnification factor: 6.28832e+03
Condition number: 4.02000e+04
EMF with b=2: 6.28832e+03
EMF with b=3: 4.26227e+03
EMF with b=4: 6.28832e+03
EMF with b=5: 1.92966e+03

n=300
BE_norm: 1.57743e-08
FE_norm: 3.03026e-09
Error magnification factor: 8.67338e+03
Condition number: 9.03000e+04
EMF with b=2: 8.67338e+03
EMF with b=3: 4.51704e+03
EMF with b=4: 8.67338e+03
EMF with b=5: 1.82113e+04

n=400
BE_norm: 5.13101e-08
FE_norm: 4.48038e-09
Error magnification factor: 7.00304e+03
Condition number: 1.60400e+05
EMF with b=2: 7.00304e+03
EMF with b=3: 2.11649e+04
EMF with b=4: 7.00304e+03
EMF with b=5: 1.48419e+04

n=500
BE_norm: 2.51166e-08
FE_norm: 9.63300e-09
Error magnification factor: 4.80373e+04
Condition number: 2

4. 对于 $A_{ij}=\sqrt{(i-j)^2+n/10}$ 的矩阵解答3中的问题。

In [67]:
makeA4 = lambda n: np.array([[np.sqrt((row-col)**2+n/10.) for row in range(1, n+1)] for col in range(1, n+1)])
for i in range(1, 6):
    Solve(makeA4, i*100, False)
    FindOtherEMF(makeA4, i*100)


n=100
BE_norm: 4.18368e-11
FE_norm: 8.90967e-08
Error magnification factor: 1.05946e+07
Condition number: 6.18431e+07
EMF with b=2: 1.05946e+07
EMF with b=3: 4.39441e+06
EMF with b=4: 1.05946e+07
EMF with b=5: 1.31844e+07

n=200
BE_norm: 5.85715e-10
FE_norm: 2.57075e-05
Error magnification factor: 8.75721e+08
Condition number: 1.29066e+10
EMF with b=2: 8.75721e+08
EMF with b=3: 1.67482e+09
EMF with b=4: 8.75721e+08
EMF with b=5: 9.69452e+08

n=300
BE_norm: 2.69210e-09
FE_norm: 2.11380e-03
Error magnification factor: 3.52789e+10
Condition number: 6.20392e+11
EMF with b=2: 3.52789e+10
EMF with b=3: 7.49658e+10
EMF with b=4: 3.52789e+10
EMF with b=5: 3.82415e+10

n=400
BE_norm: 4.49654e-09
FE_norm: 4.05175e-02
Error magnification factor: 7.20055e+11
Condition number: 1.47532e+13
EMF with b=2: 7.20055e+11
EMF with b=3: 5.91812e+11
EMF with b=4: 7.20055e+11
EMF with b=5: 9.44430e+11

n=500
BE_norm: 6.82485e-09
FE_norm: 7.03160e-01
Error magnification factor: 1.28673e+13
Condition number: 2

5. n取多少时，问题1中的解没有正确的有效数字？

In [80]:
# 姑且认为前向误差的无穷范数大于1及没有有效数字了
Solve(makeA, 12)
# TODO: 答案是13，为什么？


n=12
x:
 [ 0.99999999  1.00000205  0.999885    1.00238414  0.97585767  1.13728166
  0.52745193  2.02086771 -0.39107924  2.15976778  0.46015844  1.10742348]
BE_norm: 1.77636e-15
FE_norm: 1.39108e+00
Error magnification factor: 6.07537e+15
Condition number: 1.91350e+17


6. 使用2.1.1的程序（高斯消元法）解答例2.13（见下）中的第2和第3中方法的双精度实现，并和课本中的理论解进行比较。
$$
\begin{aligned}
10^{-20}x_1+x_2&=1\\
x_1+2x_2&=4
\end{aligned}
$$

In [107]:
# 1e-20受精度限制，暂时用1e-9代替一下
A = np.array([[1e-9, 1], [1, 2]])
b = np.array([1., 4])
x = NaiveGaussinElimination(A, b, True)
print(f'Condition number: {npnorm(A, np.inf)*npnorm(np.linalg.inv(A), np.inf):.5e}')

A = np.array([[1., 2], [1e-9, 1]])
b = np.array([4., 1])
x = NaiveGaussinElimination(A, b, True)
print(f'Condition number: {npnorm(A, np.inf)*npnorm(np.linalg.inv(A), np.inf):.5e}')

x:
 [2.00000005 1.        ]
Condition number: 9.00000e+00
x:
 [2. 1.]
Condition number: 9.00000e+00
